In [1]:
import argparse
from datetime import datetime
from functools import partial
import itertools
import time
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt

import netket as nk
from netket.operator.fermion import destroy as c
from netket.operator.fermion import create as cdag
from netket.operator import FermionOperator2nd
from netket.experimental.operator import ParticleNumberConservingFermioperator2nd

from src.qm_utils.lattice.lattice import Lattice2D
from src.netket_compat import get_sector_constraints
from src.acband import acband_form_factors, interaction_matrix, K_func1, interaction_hamiltonian_terms

from brillouin_zones import construct_brillouin_zones

/home/cwpark/acband-laughlin-instability/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


∣NK⟩ Tip: You can use flax.linen, flax.nnx and equinox to define neural networks.

In [2]:
N_s = 27
N_f = 20

In [3]:
sqrt3 = 3.0 ** 0.5

lB = 1.0
a_M = (((4 * np.pi) / sqrt3) ** 0.5) * lB
# a_M = 1
# lB = ((sqrt3 / (4 * m * np.pi)) ** 0.5) * a_M
     
# fourier_resolution = 128
fourier_resolution = 256
G_radius = 128
V1 = 1.0
# v1 = 3 * V1 * (a_M ** 4) / (4 * np.pi)
v1 = 3 * V1 * (a_M ** 4) / (8 * np.pi) # ????? 4 pi -> 8 pi


# Lattice and Brillouin zones
e1 = np.array([1, 0])
e2 = np.array([0, 1])
a1 = a_M * e2
a2 = a_M * ((-sqrt3 / 2) * e1 + (1 / 2) * e2)
lattice = Lattice2D(np.stack([a1, a2]))
recip_lattice = lattice.reciprocal()

bz = construct_brillouin_zones(lattice)

bz_N_s = bz[N_s]

In [4]:
constraints = get_sector_constraints(bz_N_s, N_f)
hilbs = [
    nk.hilbert.SpinOrbitalFermions(
        n_orbitals=N_s, n_fermions=N_f, constraint=constraint
    ) for constraint in constraints
]

for k_index, sector in enumerate(hilbs):
    print(f"Constructing Sector {k_index}:")
    print(f"  Total Momentum: {bz_N_s[k_index]}")
    print(f"  Dimension: {sector.n_states}")
 
b1, b2 = lattice.reciprocal_lattice_vectors
b3 = -(b1 + b2)

Constructing Sector 0:
  Total Momentum: [ 0.         -1.03674687]
  Dimension: 32890
Constructing Sector 1:
  Total Momentum: [ 0.44892456 -0.77756015]
  Dimension: 32890
Constructing Sector 2:
  Total Momentum: [ 0.89784912 -0.51837343]
  Dimension: 32890
Constructing Sector 3:
  Total Momentum: [-0.44892456 -0.77756015]
  Dimension: 32890
Constructing Sector 4:
  Total Momentum: [ 0.         -0.51837343]
  Dimension: 32890
Constructing Sector 5:
  Total Momentum: [ 0.44892456 -0.25918672]
  Dimension: 32890
Constructing Sector 6:
  Total Momentum: [0.89784912 0.        ]
  Dimension: 32890
Constructing Sector 7:
  Total Momentum: [-0.89784912 -0.51837343]
  Dimension: 32890
Constructing Sector 8:
  Total Momentum: [-0.44892456 -0.25918672]
  Dimension: 32890
Constructing Sector 9:
  Total Momentum: [0. 0.]
  Dimension: 32890
Constructing Sector 10:
  Total Momentum: [0.44892456 0.25918672]
  Dimension: 32890
Constructing Sector 11:
  Total Momentum: [0.89784912 0.51837343]
  Dimensi

In [5]:
b1, b2 = lattice.reciprocal_lattice_vectors
b3 = -(b1 + b2)

K_func_args = (0.8, b1, b2, b3)
K_func = partial(K_func1, args=K_func_args)

G_coords, ac_ff = acband_form_factors(
    bz[N_s],
    lB,
    K_func,
    fourier_resolution,
    G_radius=G_radius,
    pbar=True
)

Computing AC band form factors: 100%|██████████| 66049/66049 [00:00<00:00, 86420.51it/s]


In [6]:
G_vecs = recip_lattice.get_points(G_coords)
start_idx = 1
end_idx = 2 * G_radius
G_vecs_slice = G_vecs[start_idx:end_idx, start_idx:end_idx]

def V(q):
    return -v1 * np.linalg.norm(q, axis=-1) ** 2

int_mat = interaction_matrix(
    bz_N_s,
    G_coords,
    ac_ff,
    V
)

In [7]:
hamiltonians = []
for sector_index, sector in enumerate(hilbs):
    H = 0.0
    for k, p, q in itertools.product(range(N_s), repeat=3):
        H_k1_k2_k3_k4 = int_mat[k, p, q]
        k1 = bz_N_s.sum(k, q) # k + q
        k2 = bz_N_s.sub(p, q) # p - q
        k3 = p
        k4 = k
        c_dag_k1 = cdag(sector, k1)
        c_dag_k2 = cdag(sector, k2)
        c_k3 = c(sector, k3)
        c_k4 = c(sector, k4)
        H += complex(H_k1_k2_k3_k4) * (c_dag_k1 @ c_dag_k2 @ c_k3 @ c_k4)
    H = ParticleNumberConservingFermioperator2nd.from_fermionoperator2nd(H)
    hamiltonians.append(H)

In [8]:
hamiltonians_new = []
for sector_index, sector in enumerate(hilbs):
    terms, weights = interaction_hamiltonian_terms(bz_N_s, int_mat)
    H = FermionOperator2nd(
        sector,
        terms,
        weights
    )
    H = ParticleNumberConservingFermioperator2nd.from_fermionoperator2nd(H)
    hamiltonians_new.append(H)

In [9]:
for idx, (H_old, H_new) in enumerate(zip(hamiltonians, hamiltonians_new)):
    print(f"Comparing Sector {idx}:")
    print(H_old == H_new)

Comparing Sector 0:
True
Comparing Sector 1:
True
Comparing Sector 2:
True
Comparing Sector 3:
True
Comparing Sector 4:
True
Comparing Sector 5:
True
Comparing Sector 6:
True
Comparing Sector 7:
True
Comparing Sector 8:
True
Comparing Sector 9:
True
Comparing Sector 10:
True
Comparing Sector 11:
True
Comparing Sector 12:
True
Comparing Sector 13:
True
Comparing Sector 14:
True
Comparing Sector 15:
True
Comparing Sector 16:
True
Comparing Sector 17:
True
Comparing Sector 18:
True
Comparing Sector 19:
True
Comparing Sector 20:
True
Comparing Sector 21:
True
Comparing Sector 22:
True
Comparing Sector 23:
True
Comparing Sector 24:
True
Comparing Sector 25:
True
Comparing Sector 26:
True


In [ ]:
from src.qm_utils.fermion.fermion_utils import (
    bitset_to_array,
    array_to_bitset
)

ModuleNotFoundError: No module named 'qm_utils'

In [ ]:
H.get_conn_flattened

In [ ]:
H.to_sparse()